In [23]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [24]:
dim(df_finale)

[1] 1586   74

In [25]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [26]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [27]:
dim(test)
dim(train)

[1] 476 175

[1] 1110  175

In [28]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    555     555 

## Model

In [29]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [30]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [31]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [32]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [33]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

1110 samples
 174 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1110, 1110, 1110, 1110, 1110, 1110, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.6426262  0.2977786
  1       0.001  0.50  0.7110971  0.4227196
  1       0.001  1.00  0.7298460  0.4595403
  1       0.010  0.25  0.7389685  0.4776266
  1       0.010  0.50  0.7391583  0.4778969
  1       0.010  1.00  0.7361211  0.4717941
  1       0.100  0.25  0.7309493  0.4616534
  1       0.100  0.50  0.7251423  0.4501389
  1       0.100  1.00  0.7211108  0.4420743
  2       0.001  0.25  0.7126753  0.4258445
  2       0.001  0.50  0.7297612  0.4593176
  2       0.001  1.00  0.7392336  0.4781786
  2       0.010  0.25  0.7415268  0.4826975
  2       0.010  0.50  0.7372941  0.4743188
  2       0.010  1.00  0.7342796  0.4682961
  2       0.100  0.

In [34]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

1110 samples
 174 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1110, 1110, 1110, 1110, 1110, 1110, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7461929  0.4924109
  0.50  0.7479625  0.4957936
  1.00  0.7487482  0.4973926

Tuning parameter 'sigma' was held constant at a value of 0.02040865
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.02040865 and C = 1.


In [35]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

1110 samples
 174 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 1110, 1110, 1110, 1110, 1110, 1110, ... 
Resampling results:

  Accuracy   Kappa    
  0.7066339  0.4123976

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [36]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.6878173 0.7377451 0.7518610 0.7485360 0.7650000 0.7860577    0
Radial 0.7195122 0.7395577 0.7518428 0.7487482 0.7617801 0.7763819    0
Linear 0.6634845 0.6945107 0.7025000 0.7066339 0.7177616 0.7384615    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.3748936 0.4721218 0.5044135 0.4965738 0.5253624 0.5705869    0
Radial 0.4385702 0.4789865 0.5058896 0.4973926 0.5230372 0.5494645    0
Linear 0.3238764 0.3896160 0.4060098 0.4123976 0.4347702 0.4763033    0


# Prediction

In [48]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData)


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     107      64
   NotFast   87     218
                                          
               Accuracy : 0.6828          
                 95% CI : (0.6389, 0.7244)
    No Information Rate : 0.5924          
    P-Value [Acc > NIR] : 2.922e-05       
                                          
                  Kappa : 0.3307          
                                          
 Mcnemar's Test P-Value : 0.0734          
                                          
            Sensitivity : 0.5515          
            Specificity : 0.7730          
         Pos Pred Value : 0.6257          
         Neg Pred Value : 0.7148          
             Prevalence : 0.4076          
         Detection Rate : 0.2248          
   Detection Prevalence : 0.3592          
      Balanced Accuracy : 0.6623          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5515464            0.7730496            0.6257310 
      Neg Pred Value            Precision               Recall 
           0.7147541            0.6257310            0.5515464 
                  F1           Prevalence       Detection Rate 
           0.5863014            0.4075630            0.2247899 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.6622980

In [49]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData)


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     123      48
   NotFast   75     230
                                          
               Accuracy : 0.7416          
                 95% CI : (0.6998, 0.7804)
    No Information Rate : 0.584           
    P-Value [Acc > NIR] : 5.271e-13       
                                          
                  Kappa : 0.4575          
                                          
 Mcnemar's Test P-Value : 0.01906         
                                          
            Sensitivity : 0.6212          
            Specificity : 0.8273          
         Pos Pred Value : 0.7193          
         Neg Pred Value : 0.7541          
             Prevalence : 0.4160          
         Detection Rate : 0.2584          
   Detection Prevalence : 0.3592          
      Balanced Accuracy : 0.7243          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6212121            0.8273381            0.7192982 
      Neg Pred Value            Precision               Recall 
           0.7540984            0.7192982            0.6212121 
                  F1           Prevalence       Detection Rate 
           0.6666667            0.4159664            0.2584034 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.7242751

In [50]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData)


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     119      52
   NotFast   72     233
                                          
               Accuracy : 0.7395          
                 95% CI : (0.6976, 0.7784)
    No Information Rate : 0.5987          
    P-Value [Acc > NIR] : 8.371e-11       
                                          
                  Kappa : 0.4483          
                                          
 Mcnemar's Test P-Value : 0.08796         
                                          
            Sensitivity : 0.6230          
            Specificity : 0.8175          
         Pos Pred Value : 0.6959          
         Neg Pred Value : 0.7639          
             Prevalence : 0.4013          
         Detection Rate : 0.2500          
   Detection Prevalence : 0.3592          
      Balanced Accuracy : 0.7203          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6230366            0.8175439            0.6959064 
      Neg Pred Value            Precision               Recall 
           0.7639344            0.6959064            0.6230366 
                  F1           Prevalence       Detection Rate 
           0.6574586            0.4012605            0.2500000 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.7202903

In [51]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [52]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 171 controls (TestClasses fast) < 305 cases (TestClasses NotFast).
Area under the curve: 0.7367